In [1]:
import pandas as pd
from numpy import nan, ndarray
from pandas import DataFrame
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel, euclidean_distances
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    FunctionTransformer,
    MultiLabelBinarizer,
    OneHotEncoder,
)
from sklearn.manifold import TSNE
import plotly.express as px

In [2]:
df = pd.read_parquet("data/01_raw/movies_dataset_2025-05-07.parquet")

In [3]:
use_features: list[str] = [
    "title",
    "original_language",
    "popularity",
    "vote_average",
    "vote_count",
    "is_popular",
    "runtime",
    "budget",
    "revenue",
    "genres",
    "spoken_languages",
]

df = df[use_features]

In [4]:
df.drop_duplicates(
    subset=[
        "original_language",
        "popularity",
        "vote_average",
        "vote_count",
        "is_popular",
        "runtime",
        "budget",
        "revenue",
    ],
    inplace=True,
)

In [5]:
cat_cols = ["original_language"]
df[cat_cols] = df[cat_cols].astype("category")

In [6]:
df["is_popular"] = df["is_popular"].astype("int8")

In [7]:
# ruff: noqa: RUF001
spoken_languages_mappings = {
    "Français": "European (Romance)",
    "Español": "European (Romance)",
    "English": "European (Germanic)",
    "Deutsch": "European (Germanic)",
    "हिन्दी": "South Asian",
    "广州话 / 廣州話": "East Asian",
    "日本語": "East Asian",
    "Italiano": "European (Romance)",
    "Pусский": "European (Slavic)",
    "Nederlands": "European (Germanic)",
    "isiZulu": "African",
    "ภาษาไทย": "Southeast Asian",
    "普通话": "East Asian",
    "Bahasa indonesia": "Southeast Asian",
    "": "Unknown/Other",
    "தமிழ்": "South Asian",
    "suomi": "European (Other)",
    "한국어/조선말": "East Asian",
    "български език": "European (Slavic)",
    "Català": "European (Romance)",
    "Türkçe": "Middle Eastern/Central Asian",
    "Português": "European (Romance)",
    "Norsk": "European (Germanic)",
    "Dansk": "European (Germanic)",
    "svenska": "European (Germanic)",
    "Lietuvių": "European (Other)",
    "Polski": "European (Slavic)",
    "తెలుగు": "South Asian",
    "עִבְרִית": "Middle Eastern/Central Asian",
    "Український": "European (Slavic)",
    "Latin": "European (Other)",  # Could also be considered Historical
    "?????": "Unknown/Other",
    "No Language": "Unknown/Other",
    "اردو": "South Asian",
    "العربية": "Middle Eastern/Central Asian",
    "Română": "European (Romance)",
    "Íslenska": "European (Germanic)",
    "Magyar": "European (Other)",
    "فارسی": "Middle Eastern/Central Asian",
    "Bahasa melayu": "Southeast Asian",
    "Galego": "European (Romance)",
    "ქართული": "European (Other)",  # Kartvelian is a unique family, grouped here for simplicity
    "euskera": "European (Other)",  # Language Isolate, grouped here
    "Èdè Yorùbá": "African",
    "Wolof": "African",
    "Gaeilge": "European (Other)",  # Celtic, grouped here
    "Hrvatski": "European (Slavic)",
    "ελληνικά": "European (Other)",  # Hellenic, grouped here
    "Slovenčina": "European (Slavic)",
    "πੰਜਾਬੀ": "South Asian",
    "Český": "European (Slavic)",
    "Tiếng Việt": "Southeast Asian",
    "Fulfulde": "African",
    "қазақ": "Middle Eastern/Central Asian",
    "Esperanto": "Unknown/Other",  # Constructed language
    "Èʋegbe": "African",
    "বাংলা": "South Asian",
    "پښتو": "Middle Eastern/Central Asian",
    "shqip": "European (Other)",  # Albanian, grouped here
    "Srpski": "European (Slavic)",
    "Afrikaans": "European (Germanic)",
    "Kiswahili": "African",
    "Eesti": "European (Other)",  # Uralic, grouped here
    "Slovenščina": "European (Slavic)",
    "Bamanankan": "African",
    "Azərbaycan": "Middle Eastern/Central Asian",
    "Bosanski": "European (Slavic)",
    "සිංහල": "South Asian",
    "Latviešu": "European (Other)",  # Baltic, grouped here
    "Malti": "Middle Eastern/Central Asian",
    nan: "Unknown/Other",
}

In [8]:
original_language_mappings = {
    "fr": "European (Romance)",
    "es": "European (Romance)",
    "en": "European (Germanic)",
    "te": "South Asian",
    "de": "European (Germanic)",
    "hi": "South Asian",
    "ja": "East Asian",
    "nl": "European (Germanic)",
    "th": "Southeast Asian",
    "id": "Southeast Asian",
    "ht": "European (Romance)",
    "it": "European (Romance)",
    "ta": "South Asian",
    "ml": "South Asian",
    "fi": "European (Other)",
    "ko": "East Asian",
    "bg": "European (Slavic)",
    "ca": "European (Romance)",
    "pt": "European (Romance)",
    "tr": "Middle Eastern/Central Asian",
    "no": "European (Germanic)",
    "tl": "Southeast Asian",
    "da": "European (Germanic)",
    "zu": "African",
    "sv": "European (Germanic)",
    "pl": "European (Slavic)",
    "uk": "European (Slavic)",
    "zh": "East Asian",
    "ru": "European (Slavic)",
    "kn": "South Asian",
    "xx": "Unknown/Other",
    "cn": "East Asian",
    "ar": "Middle Eastern/Central Asian",
    "hu": "European (Other)",
    "fa": "Middle Eastern/Central Asian",
    "mn": "East Asian",
    "yo": "African",
    "ro": "European (Romance)",
    "sk": "European (Slavic)",
    "jv": "Southeast Asian",
    "cs": "European (Slavic)",
    "ur": "South Asian",
    "pa": "South Asian",
    "is": "European (Germanic)",
    "hr": "European (Slavic)",
    "vi": "Southeast Asian",
    "lv": "European (Other)",
    "km": "Southeast Asian",
    "ms": "Southeast Asian",
    "kk": "Middle Eastern/Central Asian",
    "ka": "European (Other)",
    "ga": "European (Other)",
    "xh": "African",
    "ig": "African",
    "el": "European (Other)",
    "bn": "South Asian",
    "tt": "Middle Eastern/Central Asian",
    "gl": "European (Romance)",
    "mk": "European (Slavic)",
    "bo": "East Asian",
    "dz": "South Asian",
    "he": "Middle Eastern/Central Asian",
    "sr": "European (Slavic)",
    "ff": "African",
    "gu": "South Asian",
    "ab": "European (Other)",
    "et": "European (Other)",
    "kl": "European (Other)",
    "lt": "European (Other)",
    "se": "European (Other)",
    "eu": "European (Other)",
    "bs": "European (Slavic)",
    "lb": "European (Germanic)",
    "mi": "Southeast Asian",
    "hy": "Middle Eastern/Central Asian",
    "su": "Southeast Asian",
    "mt": "Middle Eastern/Central Asian",
    "sl": "European (Slavic)",
}

In [9]:
def get_features_names(_, feature_names) -> ndarray:
    return feature_names


def map_lang(X: DataFrame, col: str, mappings: dict[str, str]) -> DataFrame:
    """
    Map language codes to broader categories.
    """
    X = X.copy()
    if X[col].dtype.name == "object":  # if it's a list
        X[col] = X[col].apply(lambda x: [mappings.get(item, "Unknown/Other") for item in x])
        return X
    X[col] = X[col].map(mappings)
    return X

In [10]:
class MultiLabelBinarizerTransformer(BaseEstimator, TransformerMixin):
    """A custom transformer to apply MultiLabelBinarizer within a scikit-learn pipeline.

    This transformer is designed to be used with `ColumnTransformer` on a single
    column of a pandas DataFrame that contains lists of labels (multi-label data).
    It wraps the functionality of `sklearn.preprocessing.MultiLabelBinarizer` and
    provides a `get_feature_names_out` method compatible with scikit-learn pipelines.
    """

    def __init__(self):
        """Initializes the MultiLabelBinarizerTransformer."""
        self.mlb = MultiLabelBinarizer()

    def fit(self, X, y=None):
        """Fits the MultiLabelBinarizer on the input data.

        Args:
            X: A pandas DataFrame slice with one column containing lists of labels.
            y: Ignored.

        Returns:
            self: Returns the instance itself.
        """
        self.mlb.fit(X.iloc[:, 0])
        return self

    def transform(self, X):
        """Transforms the input data using the fitted MultiLabelBinarizer.

        Args:
            X: A pandas DataFrame slice with one column containing lists of labels.

        Returns:
            numpy.ndarray: A sparse matrix representing the binarized labels.
        """
        # Transform the values of the column
        return self.mlb.transform(X.iloc[:, 0])

    def get_feature_names_out(self, input_features=None):
        """Gets the output feature names after binarization.

        Args:
            input_features: Ignored.

        Returns:
            list: A list of strings representing the output feature names (the labels).
        """
        # Return the classes learned by the fitted MultiLabelBinarizer
        return self.mlb.classes_.tolist()

In [11]:
num_cols: list[str] = [
    "popularity",
    "vote_average",
    "vote_count",
    "runtime",
    "budget",
    "revenue",
    "is_popular",
]
cat_cols: list[str] = ["original_language"]
multi_label_cat_cols: list[str] = ["genres", "spoken_languages"]

In [12]:
num_pipe = Pipeline(steps=[("imputer", SimpleImputer(strategy="median"))])
cat_pipe = Pipeline(
    steps=[
        (
            "language mapper",
            FunctionTransformer(
                map_lang,
                kw_args={
                    "col": "original_language",
                    "mappings": original_language_mappings,
                },
                feature_names_out=get_features_names,
            ),
        ),
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("one-hot", OneHotEncoder(drop="first")),
    ]
)

multi_label_genres_pipe = Pipeline(steps=[("binarizer", MultiLabelBinarizerTransformer())])

multi_label_spoken_languages_pipe = Pipeline(
    steps=[
        (
            "language mapper",
            FunctionTransformer(
                map_lang,
                kw_args={
                    "col": "spoken_languages",
                    "mappings": spoken_languages_mappings,
                },
                feature_names_out=get_features_names,
            ),
        ),
        ("binarizer", MultiLabelBinarizerTransformer()),
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", num_pipe, num_cols),
        ("cat", cat_pipe, cat_cols),
        ("genres", multi_label_genres_pipe, ["genres"]),
        ("spoken_languages", multi_label_spoken_languages_pipe, ["spoken_languages"]),
    ],
)

In [13]:
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 ['popularity', 'vote_average', 'vote_count',
                                  'runtime', 'budget', 'revenue',
                                  'is_popular']),
                                ('cat',
                                 Pipeline(steps=[('language mapper',
                                                  FunctionTransformer(feature_names_out=<function get_features_names at 0x7c5e3c345bc0>,
                                                                      func=<function map_lang at 0x7c5e3c345b20...
                                                                                            'Italiano': 'European '
                                                                                                        '(Romance)',
                                                                                            'Kiswahili': 'African',
                                                                                            'Latin': 'European '
                                                                                                     '(Other)',
                                                                                            'Latviešu': 'European '
                                                                                                        '(Other)',
                                                                                            'Lietuvių': 'European '
                                                                                                        '(Other)',
                                                                                            'Magyar': 'European '
                                                                                                      '(Other)',
                                                                                            'Malti': 'Middle '
                                                                                                     'Eastern/Central '
                                                                                                     'Asian',
                                                                                            'Nederlands': 'European '
                                                                                                          '(Germanic)',
                                                                                            'No Language': 'Unknown/Other', ...}})),
                                                 ('binarizer',
                                                  MultiLabelBinarizerTransformer())]),
                                 ['spoken_languages'])])

In [14]:
preprocessed = preprocessor.fit_transform(df)
feature_names = preprocessor.get_feature_names_out()
preprocessed = pd.DataFrame(preprocessed, columns=feature_names)

In [15]:
# This cell is for later on visualization purposes.
tsne = TSNE(n_components=2, random_state=42)
preprocessed_tsne = tsne.fit_transform(preprocessed)

In [16]:
def get_recommendations(
    title: str, sim_matriX: ndarray, df: DataFrame, sim_movies: int = 10
) -> DataFrame:
    try:
        idx = df.index[df["title"] == title].tolist()[0]
    except IndexError:
        print(f"No movie found with title: {title}")
        return DataFrame()
    sim_scores: list = list(enumerate(sim_matriX[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1 : sim_movies + 1]
    movies_idx: list = [i[0] for i in sim_scores]
    return df.iloc[movies_idx]

## Trying different vector distances algorithms
Now we'll try with different vector distances algorithms, in this case: cosine similarity, linear_kernel and euclidean distances.

- Cosine similarity and linear_kernel are heavily used when we want to get a similar "profile" and the magnitude of the vectores is not that important; movies with similar genres can be recommended even if they don't have the same revenue or budget.

- Euclidean distance: Is much more sensible to the magnitude. This could be usefull if we want to find movies more alike in all tearms: budget, revenue, genres, spoken languages, so on and forth.  

In [17]:
cosine_sim = cosine_similarity(preprocessed, preprocessed)
linear_sim = linear_kernel(preprocessed, preprocessed)
euclidean_sim = euclidean_distances(preprocessed, preprocessed)

In [18]:
target_movie: str = "The Nun II"

In [19]:
df[df["title"] == target_movie]

,title,original_language,popularity,vote_average,vote_count,is_popular,runtime,budget,revenue,genres,spoken_languages
2582,The Nun II,en,11.7683,6.714,2163,0,110,38500000,269670590,[Terror],"[English, Français, Italiano, Português, Español]"


In [20]:
get_recommendations(target_movie, cosine_sim, df, 3)

,title,original_language,popularity,vote_average,vote_count,is_popular,runtime,budget,revenue,genres,spoken_languages
6220,K.G.F: Chapter 2,kn,4.4109,7.436,163,0,168,13000000,90410749,"[Acción, Suspense, Aventura, Crimen]",[?????]
325,Moana 2,en,107.7958,7.079,2326,1,100,150000000,1059544057,"[Animación, Aventura, Familia, Comedia]",[English]
4259,Post Truth,zh,0.6436,7.100,37,0,112,14506629,102500000,"[Comedia, Acción]",[普通话]


In [21]:
get_recommendations(target_movie, linear_sim, df, 3)

,title,original_language,popularity,vote_average,vote_count,is_popular,runtime,budget,revenue,genres,spoken_languages
162,Ne Zha 2,zh,26.4673,7.881,164,0,144,80000000,2150000000,"[Animación, Fantasía, Aventura]",[普通话]
7058,Spider-Man: No Way Home,en,27.0200,7.945,20730,0,148,200000000,1921847111,"[Acción, Aventura, Ciencia ficción]","[English, ]"
1013,Inside Out 2,en,45.7898,7.557,5807,1,96,200000000,1698863816,"[Animación, Aventura, Comedia, Familia]",[English]


In [22]:
get_recommendations(target_movie, euclidean_sim, df, 3)

,title,original_language,popularity,vote_average,vote_count,is_popular,runtime,budget,revenue,genres,spoken_languages
162,Ne Zha 2,zh,26.4673,7.881,164,0,144,80000000,2150000000,"[Animación, Fantasía, Aventura]",[普通话]
7058,Spider-Man: No Way Home,en,27.0200,7.945,20730,0,148,200000000,1921847111,"[Acción, Aventura, Ciencia ficción]","[English, ]"
1013,Inside Out 2,en,45.7898,7.557,5807,1,96,200000000,1698863816,"[Animación, Aventura, Comedia, Familia]",[English]


## Let's visualize the recommendations
Since the 3 algorithms are different ways of measuring vector distance and "alikeness", let's visualize the recommendations for a given movie.

In [23]:
preprocessed_tsne_df = pd.DataFrame(preprocessed_tsne, columns=['TSNE1', 'TSNE2'])
df_viz_tsne = pd.concat([df.reset_index(drop=True), preprocessed_tsne_df], axis=1)

In [24]:
target_movie_index = df_viz_tsne[df_viz_tsne['title'] == target_movie].index[0]

In [25]:
cosine_recs_titles = get_recommendations(target_movie, cosine_sim, df, 30)['title'].tolist()
linear_recs_titles = get_recommendations(target_movie, linear_sim, df, 30)['title'].tolist()
euclidean_recs_titles = get_recommendations(target_movie, euclidean_sim, df, 30)['title'].tolist()

In [26]:
def categorize_movie(row):
  """
  Categorize each movie (target movie, cosine recommendation, linear recommendation, euclidean recommendation, or other).
  A movie can belong to multiple recommendation categories.
  """
  if row['title'] == target_movie:
    return 'Target Movie'
  elif row['title'] in cosine_recs_titles and row['title'] in linear_recs_titles and row['title'] in euclidean_recs_titles:
    return 'All 3 Recommendations'
  elif row['title'] in cosine_recs_titles and row['title'] in linear_recs_titles:
    return 'Cosine & Linear'
  elif row['title'] in cosine_recs_titles and row['title'] in euclidean_recs_titles:
    return 'Cosine & Euclidean'
  elif row['title'] in linear_recs_titles and row['title'] in euclidean_recs_titles:
    return 'Linear & Euclidean'
  elif row['title'] in cosine_recs_titles:
    return 'Cosine Recommendation'
  elif row['title'] in linear_recs_titles:
    return 'Linear Recommendation'
  elif row['title'] in euclidean_recs_titles:
    return 'Euclidean Recommendation'
  else:
    return 'Other Movies'

In [27]:
df_viz_tsne['Category'] = df_viz_tsne.apply(categorize_movie, axis=1)

In [28]:
fig = px.scatter(
    df_viz_tsne,
    x='TSNE1',
    y='TSNE2',
    color='Category',
    hover_name='title', # Show movie title on hover
    title=f'Movie Recommendations Visualization for "{target_movie}"',
    width=1000,  # pixels
    height=900   # pixels
)

# Optional: Update layout for better presentation
fig.update_layout(legend_title_text='Recommendation Type')

for trace in fig.data:
  if trace.name == 'Other Movies':
    trace.marker.opacity = 0.1 # 10% opacity

fig.show()

## Final thoughts
Either use cosine similarity or linear kernel will suficese. Linear kernel if more "stable" / related recommendations are desired, cosine if more variability is needed.